<a href="https://colab.research.google.com/github/KangHwan-Cha/MyStudy/blob/main/TensorProject/Category5C__NewProject_Weekly US Retail-training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import

In [38]:
import urllib
import pandas as pd
import tensorflow as tf
# 아래 2줄 코드는 넣지 말아 주세요!!!
url = 'https://www.dropbox.com/s/eduk281didil1km/Weekly_U.S.Diesel_Retail_Prices.csv?dl=1'
urllib.request.urlretrieve(url, 'Weekly_U.S.Diesel_Retail_Prices.csv')

('Weekly_U.S.Diesel_Retail_Prices.csv',
 <http.client.HTTPMessage at 0x7f976e327730>)

In [39]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [40]:
# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [41]:
# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)   # convert dataset
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))   # need to add who tester=> separate x, y value
    return ds.batch(batch_size).prefetch(1)

In [42]:
df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv', infer_datetime_format=True, index_col='Week of', header=0)
df.head(20)

,Weekly U.S. No 2 Diesel Retail Prices Dollars per Gallon
Week of,
1994-03-21,1.106
1994-03-28,1.107
1994-04-04,1.109
1994-04-11,1.108
1994-04-18,1.105
1994-04-25,1.106
1994-05-02,1.104
1994-05-09,1.101
1994-05-16,1.099


In [43]:
# 특성 정의
N_FEATURES = len(df.columns)
N_FEATURES

1

In [44]:
data = df.values

data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# 데이터 분할
SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [45]:
x_train.shape, x_valid.shape

((1139, 1), (285, 1))

In [46]:
BATCH_SIZE = 32  # 배치사이즈
N_PAST = 10      # 과거 데이터 (X)
N_FUTURE = 10    # 미래 데이터 (Y)
SHIFT = 1        # SHIFT    # windowed_dataset의 shift value

In [47]:
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)

valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                             n_past=N_PAST, n_future=N_FUTURE,
                             shift=SHIFT)

### Modeling

In [60]:
model = tf.keras.models.Sequential([
    Conv1D(filters=8, kernel_size=3, padding='causal', activation='relu', input_shape=[N_PAST, 1]),   # N_PAST: window size
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(32, return_sequences=True)),   # many to many # x: 10 / y: 10
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(N_FEATURES)
])

### Create check point

In [61]:
checkpoint_path = 'mycheckpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_mae',
                             verbose=1)

In [62]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.Huber(),
              metrics=['mae'])

In [63]:
model.fit(train_set,
          validation_data=valid_set,
          epochs=50,
          callbacks=[checkpoint])

Epoch 1/50
     34/Unknown - 6s 12ms/step - loss: 0.0462 - mae: 0.2339
Epoch 1: val_mae improved from inf to 0.28864, saving model to mycheckpoint.ckpt
35/35 [==============================] - 7s 54ms/step - loss: 0.0458 - mae: 0.2345 - val_loss: 0.0435 - val_mae: 0.2886
Epoch 2/50
34/35 [============================>.] - ETA: 0s - loss: 0.0275 - mae: 0.1695
Epoch 2: val_mae improved from 0.28864 to 0.20006, saving model to mycheckpoint.ckpt
35/35 [==============================] - 1s 15ms/step - loss: 0.0271 - mae: 0.1695 - val_loss: 0.0216 - val_mae: 0.2001
Epoch 3/50
31/35 [=========================>....] - ETA: 0s - loss: 0.0124 - mae: 0.1165
Epoch 3: val_mae improved from 0.20006 to 0.10472, saving model to mycheckpoint.ckpt
35/35 [==============================] - 1s 14ms/step - loss: 0.0134 - mae: 0.1241 - val_loss: 0.0070 - val_mae: 0.1047
Epoch 4/50
34/35 [============================>.] - ETA: 0s - loss: 0.0073 - mae: 0.1023
Epoch 4: val_mae improved from 0.10472 to 0.04909, 

In [64]:
model.load_weights(checkpoint_path)

In [65]:
model.evaluate(valid_set)

9/9 [==============================] - 0s 10ms/step - loss: 6.6232e-04 - mae: 0.0265


[0.0006623195367865264, 0.026512226089835167]